In [9]:
import random
import string
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import simpledialog, messagebox, scrolledtext

class PasswordGenerator:
    def __init__(self, length=12, use_digits=True, use_special=True):
        self.length = length
        self.use_digits = use_digits
        self.use_special = use_special

    def generate_password(self):
        try:
            chars = list(string.ascii_letters)
            if self.use_digits:
                chars.extend(string.digits)
            if self.use_special:
                # Remove comma from special characters
                punctuation = string.punctuation.replace(',', '')
                chars.extend(punctuation)

            password = ''.join(random.choices(chars, k=self.length))
            return password
        except ValueError:
            print("Invalid value. Using default password.")
            return ''.join(random.choices(string.ascii_letters + string.digits, k=12))
        except Exception as e:
            print(f"Error: {e}. Using fallback password.")
            return "Fallback@123"

class PasswordManager:
    def __init__(self, filename='passwords.csv'):
        self.filename = filename
        try:
            with open(self.filename, 'r'):
                pass
        except FileNotFoundError:
            try:
                with open(self.filename, 'w') as f:
                    f.write('Website,Username,Password\n')
            except Exception as e:
                print(f"Error creating password file: {e}")

    def save_password(self, website, username, password):
        try:
            website = website.replace(',', '&#44;')
            username = username.replace(',', '&#44;')
            password = password.replace(',', '')  # Ensure no comma in password

            with open(self.filename, 'a') as f:
                f.write(f'{website},{username},{password}\n')
                f.flush()
            print(f"Password saved successfully for {website}.")
        except Exception as e:
            print(f"Error saving password: {e}")

    def load_passwords(self):
        try:
            print(f"Loading from: {self.filename}")
            df = pd.read_csv(self.filename, dtype=str, engine='python')
            df.replace({'&#44;': ','}, regex=True, inplace=True)
            if set(['Website', 'Username', 'Password']).issubset(df.columns):
                return df
            else:
                return pd.DataFrame(columns=['Website', 'Username', 'Password'])
        except Exception as e:
            print(f"Error loading passwords: {e}")
            return pd.DataFrame(columns=['Website', 'Username', 'Password'])

    def get_password(self, website):
        try:
            df = self.load_passwords()
            result = df[df['Website'].str.lower() == website.lower()]
            return result if not result.empty else 'No password found'
        except Exception as e:
            return f"Error: {e}"

    def delete_password(self, website, username):
        try:
            df = self.load_passwords()
            updated_df = df[~((df['Website'].str.lower() == website.lower()) & (df['Username'].str.lower() == username.lower()))]
            if len(updated_df) < len(df):
                updated_df.to_csv(self.filename, index=False)
                print(f"Password for {website} and user {username} deleted successfully.")
            else:
                print("No matching record found to delete.")
        except Exception as e:
            print(f"Error deleting password: {e}")

def handle_gui_option(selection):
    if selection == "Generate and Save Password":
        site = simpledialog.askstring("Input", "Enter site name:")
        username = simpledialog.askstring("Input", "Enter username:")
        if site and username:
            password = gen.generate_password()
            manager.save_password(site, username, password)
            messagebox.showinfo("Success", f"Password for {site} saved successfully:\n{password}")
        else:
            messagebox.showerror("Error", "Site and Username cannot be empty.")

    elif selection == "Retrieve Password":
        site = simpledialog.askstring("Input", "Enter site name:")
        if site:
            result = manager.get_password(site)
            if isinstance(result, str):
                messagebox.showinfo("Result", result)
            else:
                messagebox.showinfo("Password Retrieved", result.to_string(index=False))
        else:
            messagebox.showerror("Error", "Site name is required.")

    elif selection == "Display All Passwords":
        df = manager.load_passwords()
        display_window = tk.Toplevel(root)
        display_window.title("Saved Passwords")
        display_window.geometry("600x400")
        text_area = scrolledtext.ScrolledText(display_window, wrap=tk.WORD, font=("Courier", 10))
        text_area.pack(expand=True, fill='both')
        if df.empty:
            text_area.insert(tk.END, "No passwords saved.")
        else:
            text_area.insert(tk.END, df.to_string(index=False))

    elif selection == "Delete Password":
        site = simpledialog.askstring("Input", "Enter site name:")
        username = simpledialog.askstring("Input", "Enter username:")
        if site and username:
            manager.delete_password(site, username)
            messagebox.showinfo("Delete", f"Requested deletion for {site} - {username}.")
        else:
            messagebox.showerror("Error", "Both site name and username are required.")

gen = PasswordGenerator(length=16)
manager = PasswordManager()

root = tk.Tk()
root.title("Password Manager Dashboard")
root.geometry("600x300")

label = tk.Label(root, text="Select a Task:", font=("Arial", 14))
label.pack(pady=20)

option_var = tk.StringVar()
option_var.set("Generate and Save Password")

options = ["Generate and Save Password", "Retrieve Password", "Display All Passwords", "Delete Password"]
dropdown = tk.OptionMenu(root, option_var, *options, command=handle_gui_option)
dropdown.config(width=25)
dropdown.pack(pady=10)

root.mainloop()


Password saved successfully for Google.
Loading from: passwords.csv
